In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json

session = requests.Session()

URL = 'https://www.stockex.co.tt/listed-securities/'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
}

r = session.get(URL, headers=headers)
if r.status_code == 200 : 
    soup = bs(r.content, 'lxml')
    scrp = soup.findAll('script')

    pattern = re.compile('(var.?equities.*=)')
    data = []
    for script in scrp:
        if(len(pattern.findall(script.text))>0):
            strng_eq = script.string.replace('\n','').replace('  ','').split(';')[0].split('var equities = ')[-1]
            strng_bonds = script.string.replace('\n','').replace('  ','').split(';')[1].split('var bonds = ')[-1]
            
            equities = json.loads(strng_eq)
            bonds = json.loads(strng_bonds)
            data = equities + bonds
            break
        else:
            continue
    print('[*] Number of Equities and Bonds : ', len(data))
    print(data)
else: print('[*] No Data')


[*] Number of Equities and Bonds :  61
[{'idsector': '7', 'idsegment': '3', 'sector': 'BANKING', 'type': 'Equity', 'segment': 'FIRST-TIER', 'status': 'I', 'iname': 'FIRST CITIZENS BANK LIMITED', 'icode': 'FIRST', 'link': 'https://www.stockex.co.tt/manage-stock/first/', 'matured': 'DeListed'}, {'idsector': '7', 'idsegment': '3', 'sector': 'BANKING', 'type': 'Equity', 'segment': 'FIRST-TIER', 'status': 'A', 'iname': 'FIRSTCARIBBEAN INTERNATIONAL BANK LIMITED', 'icode': 'FCI', 'link': 'https://www.stockex.co.tt/manage-stock/fci/', 'matured': 'Active'}, {'idsector': '7', 'idsegment': '3', 'sector': 'BANKING', 'type': 'Equity', 'segment': 'FIRST-TIER', 'status': 'A', 'iname': 'FIRST CITIZENS GROUP FINANCIAL HOLDINGS LIMITED', 'icode': 'FCGFH', 'link': 'https://www.stockex.co.tt/manage-stock/fcgfh/', 'matured': 'Active'}, {'idsector': '7', 'idsegment': '3', 'sector': 'BANKING', 'type': 'Equity', 'segment': 'FIRST-TIER', 'status': 'A', 'iname': 'NCB FINANCIAL GROUP LIMITED', 'icode': 'NCBFG',

In [ ]:
import pandas as pd

Stock_Df = []
for i in data:
    print('[*] Name : ', i['iname'])
    print('[*] Code : ', i['icode'])
    print('[*] Type : ', i['type'])
    if i['type'] == 'Equity':
        stock_type = 'stock'
    else:
        stock_type = 'bond'
    stock_Code = i['icode']
    N_URL = f'https://www.stockex.co.tt/manage-{stock_type}/{stock_Code}/'
    
    r = session.get(N_URL, headers=headers)
    if r.status_code == 200 :
        try:
            df = pd.read_html(r.content)[2]
            df['stock_Name'] = i['iname']
            df['stock_Code'] = stock_Code
            df['stock_type'] = stock_type
            print('[*] Added......')
            Stock_Df.append(df)
        except Exception as e:
            print(f'[*] Error : {e} on {stock_type}|{stock_Code}')
    print('*'*50)
    
    df = pd.concat(Stock_Df)
    # Save Scraped Data
    df.to_excel('StockData.xlsx', index=False)